In [ ]:
from pathlib import Path
from PIL import Image
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import sys
from topostats.io import LoadScans

src_dir = Path("../src")
sys.path.append(str(src_dir))

from unet import dice_loss, iou_loss

In [ ]:
model_path = Path("/Users/sylvi/topo_data/eva-rna/test-running-model/eva_rna_model_40_epoch_simple_20250316.keras")
model = tf.keras.models.load_model(model_path, custom_objects={"dice_loss": dice_loss, "iou_loss": iou_loss})

In [ ]:
# load test image
# test_dir = Path("./experimenting_data/")
# test_image_file = test_dir / "image_14.npy"
# test_image = np.load(test_image_file)

test_dir = Path("/Users/sylvi/topo_data/eva-rna/test-running-model/output/processed/")
test_image_file = test_dir / "20250317_NIH_BuffA_tip10_M3.0_00027.topostats"
# test_image_file = test_dir / "20250317_NIH_BuffA_tip10_M3.0_00031.topostats"
# test_image_file = test_dir / "20250317_NIH_BuffA_tip10_M3.0_00042.topostats"
assert test_image_file.exists(), f"Test image file {test_image_file} does not exist."

loadscans = LoadScans(img_paths=[test_image_file], channel="dummy")
loadscans.get_data()
images_dict = loadscans.img_dict

test_image_data = images_dict[list(images_dict.keys())[0]]
print(test_image_data.keys())
test_image_data = test_image_data["above"]["grain_0"]
print(test_image_data.keys())
test_image = test_image_data["original_image"]
print(type(test_image))
plt.imshow(test_image)
plt.show()

In [ ]:
# resize image to right size
pil_image = Image.fromarray(test_image)
pil_image = pil_image.resize((256, 256))
image = np.array(pil_image)
# batch dimension
image = np.expand_dims(image, axis=0)
# class dimension
image = np.expand_dims(image, axis=-1)
predicted_mask = model.predict(image)
predicted_mask = np.squeeze(predicted_mask, axis=0)

plt.imshow(predicted_mask, cmap="gray")
plt.axis("off")
plt.show()

thresholded_predictd_mask = predicted_mask > 0.5
plt.imshow(thresholded_predictd_mask, cmap="gray")
plt.axis("off")
plt.show()